In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from tensorflow.keras import layers, models

carrierFreq = 2.45e9
c = 3e8
lambda_ = c / carrierFreq
k = 2 * np.pi / lambda_

def simulate_array(radii, elements_per_ring, theta0deg, phi0=0, n_theta=1024):
    theta0 = np.deg2rad(theta0deg)
    theta = np.linspace(0, 2*np.pi, n_theta, endpoint=False)
    phi = 0
    AF_az = np.zeros_like(theta, dtype=complex)
    rings = len(radii)
    per_ring_contrib = np.zeros((rings, n_theta), dtype=complex)
    for ring in range(rings):
        a = radii[ring]
        N = elements_per_ring[ring]
        if N == 0:
            continue
        phi_n = 2 * np.pi * np.arange(N) / N
        for n in range(N):
            phase = k * a * (np.sin(theta) * np.cos(phi - phi_n[n]) -
                            np.sin(theta0) * np.cos(phi0 - phi_n[n]))
            contrib = np.exp(1j * phase)
            AF_az += contrib
            per_ring_contrib[ring, :] += contrib
    AF_norm = np.abs(AF_az) / (np.max(np.abs(AF_az)) + np.finfo(float).eps)
    AF_dB = 20 * np.log10(AF_norm + np.finfo(float).eps)
    AF_dB[AF_dB < -60] = -60
    theta_deg = np.rad2deg(theta)
    return {'theta': theta, 'theta_deg': theta_deg, 'AF_norm': AF_norm,
            'AF_dB': AF_dB, 'per_ring_contrib': per_ring_contrib}

def label_from_sim(sim_res, top_k=3, min_prominence=0.02):
    resp = sim_res['AF_norm']
    peaks, props = find_peaks(resp, distance=5, prominence=min_prominence)
    if len(peaks) == 0:
        return {'angles_deg': [-1.0]*top_k, 'dominant_rings': [-1]*top_k, 'peak_values': [0.0]*top_k}
    pk_vals = resp[peaks]
    order = np.argsort(pk_vals)[::-1]
    peaks_sorted = peaks[order]
    pk_sorted = pk_vals[order]
    theta_deg = sim_res['theta_deg'][peaks_sorted]
    per_ring = sim_res['per_ring_contrib']
    rings_idx = []
    for p in peaks_sorted[:top_k]:
        amps = np.abs(per_ring[:, p])
        rings_idx.append(int(np.argmax(amps)))
    while len(rings_idx) < top_k:
        rings_idx.append(-1)
    angles = list(theta_deg[:top_k])
    while len(angles) < top_k:
        angles.append(-1.0)
    return {'angles_deg': angles, 'dominant_rings': rings_idx, 'peak_values': list(pk_sorted[:top_k])}

def save_polar_image(sim_res, out_path, dpi=100, figsize=(3,3)):
    theta = sim_res['theta']
    AF_dB = sim_res['AF_dB']
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, polar=True)
    ax.plot(theta, AF_dB, color='black')
    ax.set_rlim([-60, 0])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    fig.savefig(out_path, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

def generate_dataset(folder='dataset', n_examples=500, top_k=3, img_size=(128,128)):
    os.makedirs(folder, exist_ok=True)
    images_folder = os.path.join(folder, 'images')
    os.makedirs(images_folder, exist_ok=True)
    labels = []
    rings = 5
    base_radii = np.linspace(0.2*lambda_, 2.2*lambda_, rings)
    for i in range(n_examples):
        radii = base_radii * (1 + 0.12*(np.random.rand(rings)-0.5))
        elements_per_ring = [int(np.random.choice([2,4,5,6])) for _ in range(rings)]
        theta0deg = float(np.random.uniform(0, 360))
        sim = simulate_array(radii, elements_per_ring, theta0deg, n_theta=1024)
        lab = label_from_sim(sim, top_k=top_k, min_prominence=0.02)
        img_path = os.path.join(images_folder, f'img_{i:05d}.png')
        save_polar_image(sim, img_path, figsize=(2,2))
        labels.append({
            'image': os.path.relpath(img_path, folder),
            'radii': list(map(float, radii)),
            'elements_per_ring': list(map(int, elements_per_ring)),
            'theta0deg': theta0deg,
            'label': lab
        })
        if (i+1) % 100 == 0:
            print(f"Generated {i+1}/{n_examples}")
    with open(os.path.join(folder, 'labels.json'), 'w') as f:
        json.dump(labels, f, indent=2)
    print(f"Dataset generated in {folder} with {n_examples} examples.")

def make_model(input_shape=(128,128,1), top_k=3, n_rings=6):
    inp = layers.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inp)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    angle_out = layers.Dense(top_k, activation='sigmoid', name='angles')(x)
    ring_logits = layers.Dense(top_k * n_rings)(x)
    ring_reshape = layers.Reshape((top_k, n_rings))(ring_logits)
    ring_out = layers.Softmax(axis=-1, name='rings')(ring_reshape)
    model = models.Model(inputs=inp, outputs=[angle_out, ring_out])
    return model

def load_data(folder='dataset', img_size=(128,128), top_k=3, n_rings=6, max_examples=None):
    with open(os.path.join(folder, 'labels.json')) as f:
        labels = json.load(f)
    X = []
    y_angles = []
    y_rings = []
    count = 0
    for item in labels:
        if max_examples is not None and count >= max_examples:
            break
        img_path = os.path.join(folder, item['image'])
        img = load_img(img_path, color_mode='grayscale', target_size=img_size)
        img = img_to_array(img)
        X.append(img)
        lab = item['label']
        angles = [(a % 360)/360.0 if a >= 0 else 0.0 for a in lab['angles_deg']]
        while len(angles) < top_k:
            angles.append(0.0)
        r_onehots = []
        for r in lab['dominant_rings']:
            idx = r if r >=0 else (n_rings-1)
            vec = np.zeros(n_rings, dtype=np.float32)
            vec[idx] = 1.0
            r_onehots.append(vec)
        while len(r_onehots) < top_k:
            vec = np.zeros(n_rings, dtype=np.float32)
            vec[-1] = 1.0
            r_onehots.append(vec)
        y_angles.append(angles[:top_k])
        y_rings.append(r_onehots[:top_k])
        count += 1
    X = np.array(X, dtype=np.float32)
    y_angles = np.array(y_angles, dtype=np.float32)
    y_rings = np.array(y_rings, dtype=np.float32)
    return X, {'angles': y_angles, 'rings': y_rings}

def build_tf_datasets(X, Y, batch_size=16, val_split=0.2, shuffle=True):
    N = len(X)
    idx = np.arange(N)
    np.random.shuffle(idx)
    split = int((1 - val_split) * N)
    train_idx = idx[:split]
    val_idx = idx[split:]
    X_train, X_val = X[train_idx], X[val_idx]
    Y_train = {k: v[train_idx] for k, v in Y.items()}
    Y_val = {k: v[val_idx] for k, v in Y.items()}
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
    val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
    if shuffle:
        train_ds = train_ds.shuffle(1000)
    train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

def build_test_dataset(X_test, Y_test, batch_size=16):
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
    test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return test_ds

def compile_and_train(train_ds, val_ds, input_shape=(128,128,1), top_k=3, n_rings=6, epochs=20):
    model = make_model(input_shape=input_shape, top_k=top_k, n_rings=n_rings)
    model.compile(
        optimizer='adam',
        loss={
            'angles':'mse',
            'rings':tf.keras.losses.CategoricalCrossentropy(from_logits=False)
        },
        loss_weights={'angles':1.0, 'rings':1.0},
        metrics={
            'angles':tf.keras.metrics.MeanAbsoluteError(),
            'rings':'accuracy'
        }
    )
    model.summary()
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    return model, history

def evaluate_and_predict(model, X_test, Y_test, top_k=3, n_show=50):
    results = model.evaluate(tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(16))
    print('Evaluation:', results)
    preds = model.predict(X_test[:n_show])
    pred_angles = preds[0]*360.0
    pred_rings = np.argmax(preds[1], axis=-1)
    for i in range(n_show):
        print(f"Example {i}: predicted angles (deg): {np.round(pred_angles[i],1)}; rings: {pred_rings[i]}")

generate_dataset(folder='dataset', n_examples=2000, top_k=3, img_size=(128,128))
X, Y = load_data('dataset', img_size=(128,128), top_k=3, n_rings=6)
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train = X[:split_index]
X_test = X[split_index:]
Y_train = {'angles': Y['angles'][:split_index], 'rings': Y['rings'][:split_index]}
Y_test = {'angles': Y['angles'][split_index:], 'rings': Y['rings'][split_index:]}
train_ds, val_ds = build_tf_datasets(X_train, Y_train, batch_size=16, val_split=0.2)
model, history = compile_and_train(train_ds, val_ds, input_shape=(128,128,1), top_k=3, n_rings=6, epochs=20)
evaluate_and_predict(model, X_test, Y_test, top_k=3, n_show=50)


Generated 100/2000
Generated 200/2000
Generated 300/2000
Generated 400/2000
Generated 500/2000
Generated 600/2000
Generated 700/2000
Generated 800/2000
Generated 900/2000
Generated 1000/2000
Generated 1100/2000
Generated 1200/2000
Generated 1300/2000
Generated 1400/2000
Generated 1500/2000
Generated 1600/2000
Generated 1700/2000
Generated 1800/2000
Generated 1900/2000
Generated 2000/2000
Dataset generated in dataset with 2000 examples.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        320 │ rescaling[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ conv2d_2[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     33,024 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 18)        │      4,626 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 3, 6)      │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ angles (Dense)      │ (None, 3)         │        771 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rings (Softmax)     │ (None, 3, 6)      │          0 │ reshape[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 131,093 (512.08 KB)

 Trainable params: 131,093 (512.08 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - angles_loss: 0.0826 - angles_mean_absolute_error: 0.2484 - loss: 1.6927 - rings_accuracy: 0.3272 - rings_loss: 1.6100 - val_angles_loss: 0.0810 - val_angles_mean_absolute_error: 0.2461 - val_loss: 1.6138 - val_rings_accuracy: 0.3094 - val_rings_loss: 1.5327
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - angles_loss: 0.0805 - angles_mean_absolute_error: 0.2475 - loss: 1.6042 - rings_accuracy: 0.3453 - rings_loss: 1.5237 - val_angles_loss: 0.0824 - val_angles_mean_absolute_error: 0.2510 - val_loss: 1.6028 - val_rings_accuracy: 0.3562 - val_rings_loss: 1.5204
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - angles_loss: 0.0804 - angles_mean_absolute_error: 0.2472 - loss: 1.6026 - rings_accuracy: 0.3538 - rings_loss: 1.5222 - val_angles_loss: 0.0815 - val_angles_mean_absolute_error: 0.2483 - val_loss: 1.5962 - val_rings_accuracy: 0.3562 - val_rings_loss: 1.5147
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - angles_loss: 0